<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=c5c6f5482db011fb2358ef69f930e77eb33eed47c138727bdfe9661208a0e86d
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-24 10:41:21
-------------------
qualified stocks: 88
with latest results: 28
still star stocks: 17
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  54.95 K
Current:  1.42 C
-------------------
Today PnL: -44.90 K (-0.32%)
Current PnL: -19.96 L (-13.3%)
CY Booked + Current PnL: -7.86 L (-5.23%)
-------------------
Total profit:  2.17 L
Total loss:  -22.13 L
-------------------
Total Booked + Current PnL: 19.12 L (15.53%)
Total Booked PnL: 39.08 L (31.74%)
Curr Year Booked PnL: 12.10 L (8.54%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 87.14 L (61.49%)
Deployed:  1.23 C
Current:  1.42 C
CAGR/XIRR %: 8.27%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,VOLTAS,1530.0,2.98,60.0,X-MC,2.79,216150.0,24408.0,13358.0,0.68,12.73,6.18,19.69,99.0,1.83,1.53,20.39,XY25,NTT,AC
58,RELIANCE,1533.0,-9.91,64.0,X-LC,3.15,225451.0,15085.0,13707.0,-0.22,7.17,6.08,13.68,37.0,1.10,1.60,24.88,XY25,NTT,REFINERIES
2,ABBOTINDIA,35195.0,-9.24,43.0,X-MC,3.33,89700.0,-978.0,15886.0,-0.71,-1.08,17.71,16.44,101.0,-0.06,0.64,18.88,X40,ATH,PHARMA
78,TTKPRESTIG,770.0,95.30,47.0,M-SC,1.98,84102.0,-16675.0,16770.0,0.02,-16.55,19.94,0.09,245.0,-0.99,0.60,10.49,OX40N,NTT,DURABLES
42,ITC,452.0,-37.08,61.0,X-LC,1.97,202911.0,2773.0,18120.0,-0.24,1.39,8.93,10.44,4.0,0.15,1.44,7.03,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAMMAANCAP,326.00,-194.32,78.0,M-SC,24.37,163521.0,14301.0,129885.0,4.18,9.58,79.43,96.62,208.0,0.11,1.16,76.97,XY25,NTT,FINANCE
32,HINDZINC,730.22,31.71,56.0,M-LC,11.08,214694.0,9618.0,102946.0,2.36,4.69,47.95,54.89,52.0,0.09,1.52,29.92,X5K,ATH,METALS
73,TATAELXSI,9161.00,-22.37,55.0,H-MC,7.49,105060.0,-26775.0,69003.0,1.07,-20.31,65.68,32.03,98.0,-0.39,0.74,17.68,OX40N,NTT,IT
36,IEX,219.00,-30.62,61.0,H-SC,17.45,198122.0,-1832.0,99061.0,1.02,-0.92,50.00,48.63,115.0,-0.02,1.40,10.53,XR,NTT,MISC
74,TATAMOTORS,1065.00,-53.17,16.0,X-LC,13.41,147305.0,-129218.0,239282.0,0.98,-46.73,162.44,39.81,54.0,-0.54,1.04,3.82,XY24,NTT,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COLPAL,3767.14,-5.20,42.0,X-MC,8.16,221080.0,-42285.0,155640.0,-3.33,-16.06,70.40,43.04,84.0,-0.27,1.57,2.64,XY25,ATH,FMCG
31,HINDUNILVR,2922.00,-10.34,44.0,X-LC,7.67,251670.0,-5393.0,40519.0,-3.26,-2.10,16.10,13.67,24.0,-0.13,1.78,17.88,XY25,NTT,FMCG
22,DABUR,735.00,-10.50,46.0,X-MC,3.91,242476.0,-5592.0,113988.0,-2.24,-2.25,47.01,43.70,102.0,-0.05,1.72,11.87,XY24,BTT,FMCG
83,VALIANTORG,838.00,-287.15,41.0,H-SC,7.86,104074.0,-59531.0,180849.0,-2.02,-36.39,173.77,74.15,139.0,-0.33,0.74,33.41,XR,NTT,CHEMICALS
26,GREENPANEL,537.00,230.75,53.0,M-SC,5.37,147522.0,-35556.0,114536.0,-1.72,-19.42,77.64,43.14,230.0,-0.31,1.05,38.77,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.0,42.74,49.0,M-MC,10.03,224704.0,-258.0,167921.0,0.89,-0.11,74.73,74.53,192.0,-0.00,1.59,32.10,XY24,BTT,STEEL
36,IEX,219.0,-30.62,61.0,H-SC,17.45,198122.0,-1832.0,99061.0,1.02,-0.92,50.00,48.63,115.0,-0.02,1.40,10.53,XR,NTT,MISC
87,WIPRO,420.0,-13.79,48.0,M-LC,5.92,151354.0,409.0,109051.0,-0.07,0.27,72.05,72.51,53.0,0.00,1.07,6.31,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UJJIVANSFB,60.0,122.87,69.0,H-SC,15.26,140508.0,-1971.0,21498.0,0.27,-1.38,15.30,13.70,163.0,-0.09,1.00,65.84,OX40N,NTT,BANKS
45,KANSAINER,340.0,-66.87,59.0,H-SC,2.38,228213.0,-41454.0,77798.0,-0.73,-15.37,34.09,13.47,138.0,-0.53,1.62,16.04,XY24,NTT,PAINTS
78,TTKPRESTIG,770.0,95.30,47.0,M-SC,1.98,84102.0,-16675.0,16770.0,0.02,-16.55,19.94,0.09,245.0,-0.99,0.60,10.49,OX40N,NTT,DURABLES
73,TATAELXSI,9161.0,-22.37,55.0,H-MC,7.49,105060.0,-26775.0,69003.0,1.07,-20.31,65.68,32.03,98.0,-0.39,0.74,17.68,OX40N,NTT,IT
67,SIS,528.0,2066.84,52.0,H-SC,3.25,87452.0,-23580.0,47189.0,-0.58,-21.24,53.96,21.26,156.0,-0.50,0.62,18.16,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RECLTD,446.00,45.36,48.0,M-LC,5.69,205645.0,2475.0,39648.0,-0.27,1.22,19.28,20.74,55.0,0.06,1.46,6.83,XY25,NTT,FINANCE
32,HINDZINC,730.22,31.71,56.0,M-LC,11.08,214694.0,9618.0,102946.0,2.36,4.69,47.95,54.89,52.0,0.09,1.52,29.92,X5K,ATH,METALS
46,KPIGREEN,731.05,7.49,68.0,H-SC,5.19,128016.0,2719.0,56212.0,-0.12,2.17,43.91,47.03,141.0,0.05,0.91,60.29,MH,ATH,POWER
80,UNIONBANK,163.00,-10.08,60.0,M-LC,9.12,161829.0,20989.0,23498.0,-0.36,14.90,14.52,31.59,66.0,0.89,1.15,45.34,XY24,NTT,BANKS
87,WIPRO,420.00,-13.79,48.0,M-LC,5.92,151354.0,409.0,109051.0,-0.07,0.27,72.05,72.51,53.0,0.00,1.07,6.31,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.00,-20.50,70.0,H-MC,14.28,206692.0,26884.0,85529.0,-0.84,14.95,41.38,62.52,88.0,0.31,1.46,54.13,XR,NTT,BANKS
86,WHIRLPOOL,2270.00,-38.52,66.0,M-SC,3.82,103995.0,12496.0,66255.0,-0.91,13.66,63.71,86.07,223.0,0.19,0.74,51.02,XR,NTT,DURABLES
32,HINDZINC,730.22,31.71,56.0,M-LC,11.08,214694.0,9618.0,102946.0,2.36,4.69,47.95,54.89,52.0,0.09,1.52,29.92,X5K,ATH,METALS
46,KPIGREEN,731.05,7.49,68.0,H-SC,5.19,128016.0,2719.0,56212.0,-0.12,2.17,43.91,47.03,141.0,0.05,0.91,60.29,MH,ATH,POWER
37,INDIAMART,4810.62,-55.39,42.0,H-SC,7.15,124635.0,1299.0,130331.0,0.08,1.05,104.57,106.72,119.0,0.01,0.88,24.68,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,16.0,X-LC,13.41,147305.0,-129218.0,239282.0,0.98,-46.73,162.44,39.81,54.0,-0.54,1.04,3.82,XY24,NTT,AUTO
24,DMART,5391.45,-16.90,34.0,X-LC,10.17,101678.0,-908.0,27717.0,-0.74,-0.89,27.26,26.13,38.0,-0.03,0.72,24.46,X40N,ATH,FMCG
53,QUESS,986.00,-48.21,35.0,X-SC,38.76,51323.0,-13683.0,164613.0,-0.28,-21.05,320.74,232.18,198.0,-0.08,0.36,0.11,XY24,NTT,MISC
21,COLPAL,3767.14,-5.20,42.0,X-MC,8.16,221080.0,-42285.0,155640.0,-3.33,-16.06,70.40,43.04,84.0,-0.27,1.57,2.64,XY25,ATH,FMCG
52,PGHH,17907.65,-29.49,42.0,X-MC,3.23,204195.0,3375.0,64424.0,-0.67,1.68,31.55,33.76,80.0,0.05,1.45,6.97,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIEMENS,4671.50,-1.54,52.0,H-LC,0.81,158370.0,-27725.0,75210.0,0.56,-14.90,47.49,25.51,15.0,-0.37,1.12,16.71,AR,ATH,ELECTRICAL
42,ITC,452.00,-37.08,61.0,X-LC,1.97,202911.0,2773.0,18120.0,-0.24,1.39,8.93,10.44,4.0,0.15,1.44,7.03,X40,NTT,FMCG
85,VOLTAS,1530.00,2.98,60.0,X-MC,2.79,216150.0,24408.0,13358.0,0.68,12.73,6.18,19.69,99.0,1.83,1.53,20.39,XY25,NTT,AC
29,HAVELLS,2069.16,-5.05,51.0,X-MC,2.86,235404.0,-12327.0,87382.0,0.84,-4.98,37.12,30.30,92.0,-0.14,1.67,7.65,X40,ATH,ELECTRICAL
58,RELIANCE,1533.00,-9.91,64.0,X-LC,3.15,225451.0,15085.0,13707.0,-0.22,7.17,6.08,13.68,37.0,1.10,1.60,24.88,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-48.21,35.0,X-SC,38.76,51323.0,-13683.0,164613.0,-0.28,-21.05,320.74,232.18,198.0,-0.08,0.36,0.11,XY24,NTT,MISC
21,COLPAL,3767.14,-5.20,42.0,X-MC,8.16,221080.0,-42285.0,155640.0,-3.33,-16.06,70.40,43.04,84.0,-0.27,1.57,2.64,XY25,ATH,FMCG
3,ACC,3906.00,-38.07,48.0,X-MC,2.91,184420.0,-53531.0,206182.0,-0.62,-22.50,111.80,64.15,174.0,-0.26,1.31,3.45,XY24,BTT,CEMENT
74,TATAMOTORS,1065.00,-53.17,16.0,X-LC,13.41,147305.0,-129218.0,239282.0,0.98,-46.73,162.44,39.81,54.0,-0.54,1.04,3.82,XY24,NTT,AUTO
10,BAJAJHFL,181.50,-14.27,46.0,X-MC,5.75,180482.0,-20024.0,117187.0,-0.14,-9.99,64.93,48.45,90.0,-0.17,1.28,4.08,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-48.21,35.0,X-SC,38.76,51323.0,-13683.0,164613.0,-0.28,-21.05,320.74,232.18,198.0,-0.08,0.36,0.11,XY24,NTT,MISC
51,PAGEIND,51769.93,-26.57,44.0,X-MC,6.74,83000.0,-510.0,20542.0,0.10,-0.61,24.75,23.98,82.0,-0.02,0.59,5.17,X40,ATH,APPARELS
57,RELAXO,1176.00,-39.20,46.0,X-SC,4.68,82852.0,-62308.0,138239.0,-0.69,-42.92,166.85,52.31,136.0,-0.45,0.59,10.90,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-9.24,43.0,X-MC,3.33,89700.0,-978.0,15886.0,-0.71,-1.08,17.71,16.44,101.0,-0.06,0.64,18.88,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-35.15,47.0,X-SC,4.43,96952.0,-31718.0,77019.0,0.04,-24.65,79.44,35.20,219.0,-0.41,0.69,11.28,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-27.40,58.0,X-LC,12.34,289059.0,-56897.0,123602.0,0.06,-16.45,42.76,19.28,1.0,-0.46,2.05,6.86,X40,ATH,IT
40,INFY,2275.00,-16.73,58.0,X-LC,7.18,324143.0,11255.0,160418.0,-0.44,3.60,49.49,54.87,3.0,0.07,2.30,10.49,X40,BTT,IT
42,ITC,452.00,-37.08,61.0,X-LC,1.97,202911.0,2773.0,18120.0,-0.24,1.39,8.93,10.44,4.0,0.15,1.44,7.03,X40,NTT,FMCG
84,VBL,671.64,-18.39,52.0,X-LC,9.88,293831.0,-22011.0,134663.0,-1.07,-6.97,45.83,35.67,5.0,-0.16,2.08,5.92,X40N,ATH,FMCG
1,ABB,7934.00,-40.45,48.0,H-LC,6.32,248904.0,-12715.0,131919.0,0.08,-4.86,53.00,45.57,7.0,-0.10,1.76,5.91,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7127.78,65.0,L-SC,7.21,79305.0,-14505.0,91121.0,-1.65,-15.46,114.90,81.67,269.0,-0.16,0.56,53.58,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,48.0,L-SC,26.95,80602.0,-32947.0,73001.0,-0.24,-29.02,90.57,35.27,268.0,-0.45,0.57,97.27,XR,NTT,HOTELS
49,MASFIN,398.61,-19.60,49.0,H-SC,9.55,91530.0,-6450.0,28054.0,-0.65,-6.58,30.65,22.05,152.0,-0.23,0.65,33.05,XR,ATH,FINANCE
86,WHIRLPOOL,2270.00,-38.52,66.0,M-SC,3.82,103995.0,12496.0,66255.0,-0.91,13.66,63.71,86.07,223.0,0.19,0.74,51.02,XR,NTT,DURABLES
46,KPIGREEN,731.05,7.49,68.0,H-SC,5.19,128016.0,2719.0,56212.0,-0.12,2.17,43.91,47.03,141.0,0.05,0.91,60.29,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAMMAANCAP,326.00,-194.32,78.0,M-SC,24.37,163521.0,14301.0,129885.0,4.18,9.58,79.43,96.62,208.0,0.11,1.16,76.97,XY25,NTT,FINANCE
79,UJJIVANSFB,60.00,122.87,69.0,H-SC,15.26,140508.0,-1971.0,21498.0,0.27,-1.38,15.30,13.70,163.0,-0.09,1.00,65.84,OX40N,NTT,BANKS
68,SONACOMS,806.63,-32.86,69.0,M-SC,9.61,83554.0,-17605.0,57610.0,-0.13,-17.40,68.95,39.54,202.0,-0.31,0.59,17.82,AR,ATH,AUTO
5,ANGELONE,3033.00,15.82,68.0,X-SC,6.41,204836.0,13830.0,43876.0,-0.52,7.24,21.42,30.21,157.0,0.32,1.45,29.63,X40N,NTT,FINANCE
59,REPCOHOME,880.00,-56.04,67.0,H-SC,2.91,252886.0,-33619.0,290945.0,-0.75,-11.73,115.05,89.82,134.0,-0.12,1.79,33.80,XY24,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.41
1,25,43.79
2,50,74.77


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.42
LC    32.77
MC    22.83
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.29
X40      16.08
XY25     12.22
XR       11.26
X40N      9.31
AR        7.91
OX40N     7.64
X200      1.79
X5K       1.52
SR        1.09
MH        0.91
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.59
X-LC    21.87
X-MC    17.07
M-SC    12.73
M-LC     5.20
H-LC     4.67
X-SC     4.63
H-MC     3.87
M-MC     1.59
L-SC     1.47
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.60,-4.59,38.72
IT,12.77,-16.68,79.58
FINANCE,12.19,-9.94,59.77
MISC,6.93,-17.34,82.35
BANKS,6.61,-6.97,63.24
PAINTS,5.63,-16.19,33.52
ELECTRICAL,5.32,-9.58,48.02
INSURANCE,3.90,-0.43,37.87
AUTO,2.81,-44.87,105.33


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3297910.0,22
XR,1319803.0,14
AR,1177886.0,9
X40,777727.0,12
X40N,614617.0,8
OX40N,568927.0,10
XY25,458533.0,8
SR,267716.0,2
X5K,102946.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3390352.0,25
M-SC,1210889.0,15
X-MC,1128311.0,13
X-LC,1050912.0,13
X-SC,527069.0,6
H-MC,332531.0,3
H-LC,278869.0,3
M-LC,275143.0,4
L-SC,253461.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1169595.0      6
           AR         846252.0      5
           XR         786996.0      7
M-SC       XY24       695076.0      6
X-MC       XY24       592600.0      4
X-LC       X40        451182.0      5
           XY24       306608.0      2
X-SC       X40N       285437.0      4
H-SC       SR         267716.0      2
           OX40N      263581.0      4
X-MC       X40        249526.0      6
X-LC       X40N       211993.0      3
H-LC       AR         207129.0      2
H-MC       XY24       177999.0      1
X-MC       XY25       168998.0      2
M-MC       XY24       167921.0      1
X-SC       XY24       164613.0      1
L-SC       XR         164122.0      2
M-SC       OX40N      147004.0      4
           XY25       129885.0      1
           AR         124505.0      2
X-MC       X40N       117187.0      1
M-SC       XR         114419.0      2
M-LC       XR         109051.0      1
           X5K        102946.0      1
L-SC       OX40N       89339.0      1
H-MC       XR          85529.0      1
X-LC       XY25        81129.0      3
X-SC       X40         77019.0      1
H-LC       X200        71740.0      1
H-MC       OX40N       69003.0      1
L-MC       XR          59686.0      1
H-SC       MH          56212.0      1
M-LC       XY25        39648.0      1
L-LC       XY25        38873.0      1
M-LC       XY24        23498.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
